<a href="https://colab.research.google.com/github/QingyunSun/Create-your-Own-Dataset-AutoML/blob/main/auto_dataset_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Replace 'YOUR_OPENAI_API_KEY' with your actual OpenAI API key
api_key = ''

# Set the environment variable
os.environ['OPENAI_API_KEY'] = api_key


In [ ]:
import os
import pickle

# Set the path to the directory
dir_path = '/'

# Create a dictionary to store the pickle objects
pickle_dict = {}

# Walk through the files in the directory
for filename in os.listdir(dir_path):
    # Ensure we're working with a pickle file
    if filename.endswith('.pkl'):
        with open(os.path.join(dir_path, filename), 'rb') as f:
            # Load the pickle file and store it in the dictionary
            pickle_obj = pickle.load(f)
            pickle_dict[filename] = pickle_obj

# Now, pickle_dict holds all of your loaded pickle objects
# Key is the filename and value is the pickle object


In [ ]:
pickle_dict.keys()

dict_keys(['2.pkl', '7.pkl', '5.pkl', '11.pkl', '3.pkl', '14.pkl', '9.pkl', '12.pkl', '1B.pkl', '7A.pkl', '4.pkl', '1A.pkl', '8.pkl', '10.pkl', '9A.pkl', '1.pkl', '6.pkl', '13.pkl'])

In [ ]:
print(pickle_dict['1.pkl'].head())

  ticker                                          section1A       date  \
0   aapl   Item 1. Business \n\nCompany Background \n\nT... 2022-09-24   
1   aapl   Item 1. Business \n\nCompany Background \n\nT... 2021-09-25   
2   aapl   Item 1. Business \n\nCompany Background \n\nT... 2020-09-26   
3   aapl   Item 1. \n\nBusiness \n\n##TABLE_END Company ... 2019-09-28   
4   aapl   Item 1. \n\nBusiness \n\n##TABLE_END Company ... 2018-09-29   

       ticker_date  
0  aapl_2022-09-24  
1  aapl_2021-09-25  
2  aapl_2020-09-26  
3  aapl_2019-09-28  
4  aapl_2018-09-29  


In [ ]:
print(pickle_dict['14.pkl'].iloc[0]['section14'])

 Item 14. Principal Accountant Fees and Services 

The information required by this Item will be included in the 2023 Proxy Statement, and is incorporated herein by reference. 

Apple Inc. | 2022 Form 10-K | 54 

PART IV 




In [ ]:
!pip install langchain

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 3000,
    chunk_overlap  = 300,
    length_function = len,
    add_start_index = True,
)

texts = text_splitter.create_documents([pickle_dict['8.pkl'].iloc[0]['section8']])
print(texts[0])
print(texts[1])


page_content='Item 8. Financial Statements and Supplementary Data \n\n##TABLE_START Index to Consolidated Financial Statements Page Consolidated Statements of Operations for the years ended September 24, 2022, September 25, 2021 and September 26, 2020 \n\nConsolidated Statements of Comprehensive Income for the years ended September 24, 2022, September 25, 2021 and September 26, 2020 \n\nConsolidated Balance Sheets as of September 24, 2022 and September 25, 2021 \n\nConsolidated Statements of Shareholders&#8217; Equity for the years ended September 24, 2022, September 25, 2021 and September 26, 2020 \n\nConsolidated Statements of Cash Flows for the years ended September 24, 2022, September 25, 2021 and September 26, 2020 \n\nNotes to Consolidated Financial Statements \n\nReports of Independent Registered Public Accounting Firm \n\n##TABLE_END\n\nAll financial statement schedules have been omitted, since the required information is not applicable or is not present in amounts sufficient t

In [ ]:
len(texts)

31

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import LLMChain

# p = """
# As a capable financial analyst, your task is to review a section from Apple Inc.'s latest SEC filing.
# This section contains information about the company's financial position, its recent performance, and future plans.
# Please create a list of questions that could be answered by the content of the filing, along with their respective answers.
# These questions should focus on key points such as financial health, risk factors, executive compensation, business operations, and forward-looking statements.
# Provide concise, clear answers for each question based on the information presented in the filing.

# {filing_content}
# """

p_v2 = """
As a capable financial analyst, your task is to review a section from Apple Inc.'s latest SEC filing.
This section contains specific financial data, including key details about the company's revenue, sources of income, and profitability.
Generate a series of time-sensitive questions that specifically reference the concrete numbers and data points within the filing related to Apple's revenue streams.
For each question, provide a corresponding answer that is directly informed by the specific data in the document.
The question-answer pairs should cover areas such as revenue growth, breakdown of revenue by product or service, the impact of external factors on revenue, and how revenue performance might influence future strategy.

For the output, please format the question-answer pairs as a JSON object.
Each question-answer pair should be a separate object, with the question as the 'question' key and the answer as the 'answer' key.

{filing_content}
"""

import json

def is_valid_json(json_string):
    try:
        json.loads(json_string)
        return True
    except json.JSONDecodeError:
        return False

def generate_qa_pairs_from_filing(filing_content):
  llm = ChatOpenAI(temperature=0.1, model_name='gpt-4')
  # llm = OpenAI(temperature=0.1)
  prompt = PromptTemplate(
      input_variables=["filing_content"],
      template = p_v2
  )
  chain = LLMChain(llm=llm, prompt=prompt)
  print(chain.run(filing_content))


In [ ]:
generate_qa_pairs_from_filing(texts[0].page_content)

KeyboardInterrupt: ignored

In [ ]:
for t in texts:
  generate_qa_pairs_from_filing(t.page_content)

KeyboardInterrupt: ignored